In [2]:
import FinanceDataReader as fdr
import requests, lxml
from bs4 import BeautifulSoup
from selenium import webdriver

import yfinance as yf
from pandas_datareader import data as pdr

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time, datetime

### STOCK List&Chart

In [20]:
start = '2022.05.20'
end = '2022.05.31'

In [9]:
"AAPL" in sorted(sp500['Symbol'])

True

In [20]:
sp500 = fdr.StockListing('S&P500')

f = open('./crawl/corporate.txt', 'w')

for c in sp500['Symbol']:
    data = f"{c}\n"
    f.write(data)
    
f.close()

In [5]:
sp500[:50]['Symbol'][:10]

0     MMM
1     AOS
2     ABT
3    ABBV
4    ABMD
5     ACN
6    ATVI
7     ADM
8    ADBE
9     ADP
Name: Symbol, dtype: object

In [3]:
dr_krx = fdr.StockListing('NASDAQ')

100%|█████████████████████████████████████| 4629/4629 [00:09<00:00, 473.83it/s]


In [4]:
dr_krx[:10]

,Symbol,Name,Industry,IndustryCode
0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",571060
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,572010
2,AMZN,Amazon.com Inc,다양한 소매업,534020
3,GOOGL,Alphabet Inc Class A,소프트웨어 및 IT서비스,572010
4,TSLA,Tesla Inc,자동차 및 자동차 부품,531010
5,GOOG,Alphabet Inc Class C,소프트웨어 및 IT서비스,572010
6,FB,Meta Platforms Inc Class A,소프트웨어 및 IT서비스,572010
7,NVDA,NVIDIA Corp,반도체 및 반도체 장비,571010
8,ONBPP,Old National Bancorp 40 Depositary Shares Rep ...,은행,551010
9,PEP,PepsiCo Inc,음료,541010


In [21]:
df = fdr.DataReader('AAPL', start, end)

#df['Close'].plot()

In [22]:
df

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2022-05-20,137.59,139.09,140.70,132.61,137430000.0,0.0017
2022-05-23,143.11,137.79,143.26,137.65,116070000.0,0.0401
2022-05-24,140.36,140.80,141.97,137.33,103550000.0,-0.0192
2022-05-25,140.52,138.43,141.79,138.34,91840000.0,0.0011
2022-05-26,143.78,137.39,144.34,137.14,90350000.0,0.0232
2022-05-27,149.64,145.39,149.68,145.26,90980000.0,0.0408
2022-05-31,148.84,149.07,150.66,146.84,103320000.0,-0.0053


### Crawling

In [2]:
# options
headers = {
    "User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36"
}

# params = {
#     'q' : 'AAPL',
#     'hl' : 'en-US',
#     'gl' : 'US',
#     'ceid' : 'US%3Aen'
# }
# response = requests.get("https://news.google.com/search", headers=headers, params=params)

### title xpath 형식

//*[@id="quoteNewsStream-0-Stream"]/ul/li[1]/div/div/div[2]/h3/a/text()  
//*[@id="quoteNewsStream-0-Stream"]/ul/li[6]/div/div/div[2]/h3/a/text()  
//*[@id="quoteNewsStream-0-Stream"]/ul/li[8]/div/div/div[2]/h3/a/text()  
//*[@id="quoteNewsStream-0-Stream"]/ul/li[19]/div/div/div[2]/h3/a/text()  

In [3]:
titles = []

# selenium setting
url = "https://finance.yahoo.com/quote/AAPL?p=AAPL&.tsrc=fin-srch"
whileSeconds = 10
SCROLLTIME = 60

# protect deny
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36')

# start, end
start = datetime.datetime.now()
end = start + datetime.timedelta(seconds=whileSeconds)    

# open yahoo finance
driver = webdriver.Chrome(executable_path='/Users/parkdoyeong/Documents/Paper/STOCK2022/chromedriver', options=options)
driver.get(url)

# scroll
# excute_script method
# while True:
#     driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
#     time.sleep(whileSeconds)
    
#     if datetime.datetime.now() > end:
#         break

# scroll
# send_kes method
driver.implicitly_wait(whileSeconds)
body = driver.find_element_by_css_selector('body') 

for i in range(SCROLLTIME):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.3)

# xpath element -> string
title = driver.find_element(By.XPATH, '//*[@id="quoteNewsStream-0-Stream"]/ul')
title = title.get_attribute('innerText')
titles.append(title)

driver.close()

/var/folders/xw/rbcyfp692g3c5w_0qcls7ncm0000gn/T/ipykernel_24070/2809960807.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/parkdoyeong/Documents/Paper/STOCK2022/chromedriver', options=options)
/var/folders/xw/rbcyfp692g3c5w_0qcls7ncm0000gn/T/ipykernel_24070/2809960807.py:33: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  body = driver.find_element_by_css_selector('body')


In [19]:
len(titles[0].split('\n'))

513

In [15]:
len(titles[0].split('\n'))

1233

In [4]:
msft = yf.Ticker("MSFT")

In [12]:
msft.actions

,Dividends,Stock Splits
Date,,
1987-09-21,0.00,2.0
1990-04-16,0.00,2.0
1991-06-27,0.00,1.5
1992-06-15,0.00,1.5
1994-05-23,0.00,2.0
...,...,...
2021-05-19,0.56,0.0
2021-08-18,0.56,0.0
2021-11-17,0.62,0.0


In [49]:
data = yf.download(tickers = "AAPL", period = '1d', interval = '1h')

[*********************100%***********************]  1 of 1 completed


In [50]:
data

,Open,High,Low,Close,Adj Close,Volume
2022-06-16 09:30:00-04:00,132.080002,132.389999,130.589996,131.160004,131.160004,23170248
2022-06-16 10:30:00-04:00,131.160004,131.369995,129.889999,130.910004,130.910004,15577632
2022-06-16 11:30:00-04:00,130.899994,131.080002,130.195007,131.009995,131.009995,8704304
2022-06-16 12:30:00-04:00,131.009995,131.529999,130.669998,131.250000,131.250000,8681912
2022-06-16 13:30:00-04:00,131.255005,131.520004,129.669998,129.684998,129.684998,10110845
2022-06-16 14:30:00-04:00,129.679993,130.300003,129.289993,130.115005,130.115005,12751023
2022-06-16 15:30:00-04:00,130.115005,130.279999,129.039993,130.100006,130.100006,14625257
2022-06-16 16:00:00-04:00,130.059998,130.059998,130.059998,130.059998,130.059998,0


In [22]:
for a in data.index:
    print(a)

2022-06-09 11:00:00-04:00
2022-06-09 11:01:00-04:00
2022-06-09 11:02:00-04:00
2022-06-09 11:03:00-04:00
2022-06-09 11:04:00-04:00
2022-06-09 11:05:00-04:00
2022-06-09 11:06:00-04:00
2022-06-09 11:07:00-04:00
2022-06-09 11:08:00-04:00
2022-06-09 11:09:00-04:00
2022-06-09 11:10:00-04:00
2022-06-09 11:11:00-04:00
2022-06-09 11:12:00-04:00
2022-06-09 11:13:00-04:00
2022-06-09 11:14:00-04:00
2022-06-09 11:15:00-04:00
2022-06-09 11:16:00-04:00
2022-06-09 11:17:00-04:00
2022-06-09 11:18:00-04:00
2022-06-09 11:19:00-04:00
2022-06-09 11:20:00-04:00
2022-06-09 11:21:00-04:00
2022-06-09 11:22:00-04:00
2022-06-09 11:23:00-04:00
2022-06-09 11:24:00-04:00
2022-06-09 11:25:00-04:00
2022-06-09 11:26:00-04:00
2022-06-09 11:27:00-04:00
2022-06-09 11:28:00-04:00
2022-06-09 11:29:00-04:00
2022-06-09 11:30:00-04:00
2022-06-09 11:31:00-04:00
2022-06-09 11:32:00-04:00
2022-06-09 11:33:00-04:00
2022-06-09 11:34:00-04:00
2022-06-09 11:35:00-04:00
2022-06-09 11:36:00-04:00
2022-06-09 11:37:00-04:00
2022-06-09 1

In [71]:
for a in data.index:
    print(a)

2022-06-13 09:30:00-04:00
2022-06-13 09:31:00-04:00
2022-06-13 09:32:00-04:00
2022-06-13 09:33:00-04:00
2022-06-13 09:34:00-04:00
2022-06-13 09:35:00-04:00
2022-06-13 09:36:00-04:00
2022-06-13 09:37:00-04:00
2022-06-13 09:38:00-04:00
2022-06-13 09:39:00-04:00
2022-06-13 09:40:00-04:00
2022-06-13 09:41:00-04:00
2022-06-13 09:42:00-04:00
2022-06-13 09:43:00-04:00
2022-06-13 09:44:00-04:00
2022-06-13 09:45:00-04:00
2022-06-13 09:46:00-04:00
2022-06-13 09:47:00-04:00
2022-06-13 09:48:00-04:00
2022-06-13 09:49:00-04:00
2022-06-13 09:50:00-04:00
2022-06-13 09:51:00-04:00
2022-06-13 09:52:00-04:00
2022-06-13 09:53:00-04:00
2022-06-13 09:54:00-04:00
2022-06-13 09:55:00-04:00
2022-06-13 09:56:00-04:00
2022-06-13 09:57:00-04:00
2022-06-13 09:58:00-04:00
2022-06-13 09:59:00-04:00
2022-06-13 10:00:00-04:00
2022-06-13 10:01:00-04:00
2022-06-13 10:02:00-04:00
2022-06-13 10:03:00-04:00
2022-06-13 10:04:00-04:00
2022-06-13 10:05:00-04:00
2022-06-13 10:06:00-04:00
2022-06-13 10:07:00-04:00
2022-06-13 1

In [38]:
data = pdr.get_data_yahoo("AAPL", start="2021-01-01", end="2022-04-30")

In [40]:
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-12-31,134.740005,131.720001,134.080002,132.690002,99116600.0,131.515961
2021-01-04,133.610001,126.760002,133.520004,129.410004,143301900.0,128.264984
2021-01-05,131.740005,128.429993,128.889999,131.009995,97664900.0,129.850830
2021-01-06,131.050003,126.379997,127.720001,126.599998,155088000.0,125.479851
2021-01-07,131.630005,127.860001,128.360001,130.919998,109578200.0,129.761627
...,...,...,...,...,...,...
2022-04-25,163.169998,158.460007,161.119995,162.880005,96046400.0,162.641037
2022-04-26,162.339996,156.720001,162.250000,156.800003,95623200.0,156.569962
2022-04-27,159.789993,155.380005,155.910004,156.570007,88063200.0,156.340302
